# SHAP Values for Multi-Output Regression Models

Author: [coryroyce](https://github.com/coryroyce)

Date updated: 3/4/2021

## Create Multi-Output Regression Model

### Create Data

Import required packages

In [ ]:
import pandas as pd
from sklearn.datasets import make_regression
from keras.models import Sequential
from keras.layers import Dense

Create a custom function that generates the multi-output regression data.
Note: Creating 5 outputs/targets/labels for this example, but the method easily extends to any number or outputs.

In [ ]:
def get_dataset():
  # Create sample data with sklearn make_regression function
  X, y = make_regression(n_samples=1000, n_features=10, n_informative=7, n_targets=5, random_state=0)

  # Convert the data into Pandas Dataframes for easier maniplution and keeping stored column names
  # Create feature column names
  feature_cols = ['feature_01', 'feature_02', 'feature_03', 'feature_04', 
                  'feature_05', 'feature_06', 'feature_07', 'feature_08',
                  'feature_09', 'feature_10']
  
  df_features = pd.DataFrame(data = X, columns = feature_cols)               

  # Create lable column names and dataframe
  label_cols = ['labels_01', 'labels_02', 'labels_03', 'labels_04', 'labels_05']

  df_labels = pd.DataFrame(data = y, columns = label_cols)

  return df_features, df_labels

### Create Model

Create a Tensorflow/Keras Sequential model.

In [ ]:
def get_model(n_inputs, n_outputs):
	model = Sequential()
	model.add(Dense(32, input_dim=n_inputs, kernel_initializer='he_uniform', activation='relu'))
	model.add(Dense(n_outputs, kernel_initializer='he_uniform'))
	model.compile(loss='mae', optimizer='adam')
	return model

### Train Model

Create the dataset with the function defined above.

In [ ]:
# Create the datasets
X, y = get_dataset()

# Get the number of inputs and outputs from the dataset
n_inputs, n_outputs = X.shape[1], y.shape[1]

Load the model with function defined above.

In [ ]:
model = get_model(n_inputs, n_outputs)

Train the model

In [ ]:
model.fit(X, y, verbose=0, epochs=100)

Get model evaluation metrics to confirm training went well.

In [ ]:
model.evaluate(x = X, y = y)

### Model Prediction

Manual data can be entered but in this case, just us an arbitrary index form the feature/X data.

In [ ]:
model.predict(X.iloc[0:1,:])

## Get SHAP Values and Plots

Apply Shapley vaules to the model.

In [ ]:
!pip install shap
import shap

# print the JS visualization code to the notebook
shap.initjs()

Here we take the Keras model trained above and explain why it makes different predictions on individual samples.

Set the explainer using the Kernel Explainer (Model agnostic explainer method form SHAP).

In [ ]:
explainer = shap.KernelExplainer(model = model.predict, data = X.head(50), link = "identity")

Get the Shapley value for a single example.

In [ ]:
# Set the index of the specific example to explain
X_idx = 0

shap_value_single = explainer.shap_values(X = X.iloc[X_idx:X_idx+1,:], nsamples = 100)

Display the details of the single example

In [ ]:
X.iloc[X_idx:X_idx+1,:]

Choose the label/output/target to run individual explanations on:

Note: The dropdown menu can easily be replaced by manually setting the index on the label to explain. 

In [ ]:
import ipywidgets as widgets

In [ ]:
# Create the list of all labels for the drop down list
list_of_labels = y.columns.to_list()

# Create a list of tuples so that the index of the label is what is returned
tuple_of_labels = list(zip(list_of_labels, range(len(list_of_labels))))

# Create a widget for the labels and then display the widget
current_label = widgets.Dropdown(options=tuple_of_labels,
                              value=0,
                              description='Select Label:'
                              )

# Display the dropdown list (Note: access index value with 'current_label.value')
current_label


Plot the force plot for a single example and a single label/output/target

In [ ]:
# print the JS visualization code to the notebook
shap.initjs()

print(f'Current label Shown: {list_of_labels[current_label.value]}')

shap.force_plot(base_value = explainer.expected_value[current_label.value],
                shap_values = shap_value_single[current_label.value], 
                features = X.iloc[X_idx:X_idx+1,:]
                )

Create the summary plot for a specific output/label/target.

In [ ]:
# Note: We are limiting to the first 50 training examples since it takes time to calculate the full number of sampels
shap_values = explainer.shap_values(X = X.iloc[0:50,:], nsamples = 100)

In [ ]:
# print the JS visualization code to the notebook
shap.initjs()

print(f'Current Label Shown: {list_of_labels[current_label.value]}\n')

shap.summary_plot(shap_values = shap_values[current_label.value],
                  features = X.iloc[0:50,:]
                  ) 


Summary Plot Notes:

Based on the above summary plot we can see that Features 01, 03, and 07 are the features that have no influence on the model and could be dropped (Note that in the data setup we chose 10 features and only 7 of them had a meaningful relationship with the labels/targets). This is the big advantage of SHAP since without this we would not have much insight into which features are actually influencing our model.

The above setup with the dropdown menu allows us to choose the individual labels to explore in more detail. 

Force Plot for the first 50 individual examples.

In [ ]:
print(f'Current Label Shown: {list_of_labels[current_label.value]}\n')

# print the JS visualization code to the notebook
shap.initjs()

shap.force_plot(base_value = explainer.expected_value[current_label.value],
                shap_values = shap_values[current_label.value], 
                features = X.iloc[0:50,:]
                )

## Reference

Multi-output regression model format/build was largely based on [Deep Learning Models for Multi-Output Regression](https://machinelearningmastery.com/deep-learning-models-for-multi-output-regression/)